---
title: Homework
subtitle:  "Computational Economics (ECO309)"
format:
    html: default
    ipynb: default
---

__Name__:

__Surname__: 

After completing the following questions, send the edited notebook to `pablo.winant@polytechnique.fr`

You are allowed to use any online available resource, and to you use any julia package, but not to copy/paste any code.

Also, don't forget to comment your code and take any initiative you find useful and/or interesting. Also don't wait to be prompted to interpret your own results (or lack thereof).

## Part I : A simple CGE model

The goal of this exercise consists in solving a simple CGE model using Julia.

Model is taken from *Handbook of Computible General Equilibrium Modeling*. [^gams]

[^gams]: GAMS is a modeling software, routinely used for CGEs. You can checkout the code for the current model [here](https://www.gams.com/latest/gamslib_ml/libhtml/gamslib_splcge.html)

### The data and the social accounting matrix

In a CGE, the data about the economy's exchanges is compiled in one single "Social Accounting Matrix".
It is essentially a matrix representation of the national accounts that can be arbitrary refined based on data availability.

In this course work we consider the following simplified version. All items are in million euros.


|    SAM   |     BRD  |  MLK  |   CAP  |  LAB  |  HOH |
|:--------:|:--------:|:-----:|:------:|:-----:|:----:|
|   BRD    |          |       |        |       |  15  |
|  MLK     |          |       |        |       |  35  |
|  CAP     |     5    |   20  |        |       |      |
|  LAB     |    10    |   15  |        |       |      |
|  HOH     |          |       |    25  |   25  |      |


In this table, there are 5 sectors:

- Two Goods: BRD and MLK
- Two Factors: CAP and LAB (capital and labour)
- One Agent: HOH (Households)

Each of the rows feature a decomposition of the spending associated with each column.

For instance, the value of the production in the BRD-producing sector (column BRD) is decomposed between the payment to capital (5) and the payment to labor (10). The full capital income goes to households (25). Households spend their income between BRD (15) and MLK (35.)

@. Create a DataFrame representing the SAM matrix.

In [3]:
using DataFrames

SAM = ...

### Notations

To shorten notations while preserving readibility we associate conventional indices to each sector: $i \in \text{BRD,MLK}$, $j\in \text{BRD,MLK}$, $h \in \text{CAP,LAB}$).

To model the economy, we will assume that endowments $F_h$ are in fixed quantities (so $F_{BRD}$ and $F_{MLK}$ are two fixed parameters.)


We use the following notations:

- $X_i$   household consumption of good i 
- $F_{h,j}$ the h-th factor input by firm j
- $Z_{j}$   output of the j-th good
- $p^x_{i}$  demand price of the i-th good
- $p^z_{j}$  supply price of the i-th good
- $p^f_{h}$  the h-th factor price


Our goal is to endogenize all these variables (there are 14 of them), by constructing a system of equations corresponding to the model we describe below.

We also denote by a zero exponent, the variables at the initial steady-state. For instance, $X_{BRD}^0$ is the household consumption of good $BRD$. We assume that the SAM corresponds to values observed at the initial steady-state.

### Consumers


Households are endowed with capital (resp labour) in quantity $F_{CAP}$ resp $F_{LAB}$) that they sell to the goods producing firms at price $p^f_{CAP}$ (resp $p^f_{LAB}$).

Given prices of goods $p^x_i$, households maximize utility of consumption $$U = \prod_i X_i^{\alpha_i}$$ where $\alpha_i$ is the share parameter in the utility function (with $\sum_i \alpha_i=1$)


@. __Write down the budget constraint of households. Show that household's demand can be written as function of total gdp and price of goods as in   $$X_i  = \alpha_i\frac{ \sum_h  p^f_h F_h}{p^x_i}$$.__

@. __According to the SAM matrix, what is the value of gdp in the model? How can we calibrate the values of $\alpha_i$?__

::: {callout-note}
  $$X_i  = \alpha_i  \frac{\sum_h p^f_h F_h}{p^x_i}$$
$$\alpha_i  = \frac{p^x_i X^0_i}{\sum_j p^x_j X^0_{j}}$$
:::

### Producers

We assume that producers of final goods, use the following production function:

$$Z_j   = b_j \prod_h F_{h,j}^{\beta_{h,j}}$$

where $\beta_{h,j}$ is the share parameter in production function ($\sum_h \beta_{h,j}=1$) and $b_j$ is a scale parameter. Firms rent labour and capital at market price and are perfectly competitive.


@. __Show that factor demand function can be written as:__
$$ F_{h,j} = \beta_{h,j} \frac{p^z_j Z_j }{p^f_h} $$

@. __How can we calibrate $\beta_{h,j}$ from the data? How can we recover $b_j$?__

@. __Without loss of generality we can assume that all prices are equal to 1 in the initial equilibrium. [^footnote]
What are then the values for endowments $F_h$ consistent with the SAM? How can we calibrate the value of $b_j$?__

[^footnote]: this can be done by renormalizing the unit of the quantities.

::: {.callout-note}

$$\beta_{h,j} = \frac{F^0_{h,j}}{\sum_k F^0_{k,j}}$$
$$b_j      = \frac{Z^0_j}{ \prod_h  (F^0_{h,j})^{\beta_{h,j}} }$$

:::

### Equilibrium

In equilibrium we have the following conditions:

- demand prices of the i-th good must match supply price of the same good:
$$p^x_{i} = p^z_{i}$$

- good markets must clear:
$$ X_i   = Z_i $$

- factor markets must clear:
$$ \sum_j F_{h,j} = F_h $$

- the price of labour is taken as the numeraire $p_{LAB} = 1$ (the other prices are endogenous).

<!-- @. __Justify the steady-state identity: $Z^0_j   = \sum_h F^0_{h,j}$.__ -->


### Calibration and solution

@. __Define a structure `model`, containing the parameters of the model that can be directly calibrated from the data (i.e. $\alpha_i$,$\beta_{ij}$, $b_j$, $F_h$ ...).__

In [4]:
## Choose the structure your prefer

@. __Create an initial set of values `vars0` for the model *variables* to be solved for. These variables should be consistent with the Social Accounting Matrix.__

In [ ]:
## one suggestion consists in defining a named tuple of the form:

vars0 = (;
    X=(;
        BRD=...,
        MLK=...
    ),
    F=(;
        LAB=(;
            BRD=...,
            MLK=...
        )
    ),
    ...
)

@. __Write a function which packs the set of variable values into a julia vector. Write the reverse function to unpack a vector into a structure of the same type as vars. Check that these two functions are consistent (i.e. `pack(unpack(u)) == u`).__

In [ ]:
# input argument should be the same as vars0
function pack(vars)::Vector

    # [X_BRD, X_MLK, F_LAB_BRD, ...]
end

In [7]:
# output argument should be the same as vars0
function unpack(u::AbstractVector)

    
end

unpack (generic function with 1 method)

@. __Write a function `residuals` which check whether the model equations are satisfied for a particular guess `v0`.  The output can be a simple vector. Check the residuals for the initial guess `vars0`. They should be zeros is if the initial values and the model equations have been carefully chosen.__ 

In [ ]:
function residuals(model, v)::Vector

    # write down the model equations below
    

end

@. __Add a new method to the function `residuals`, which now takes a vector as argument (using the unpack function). Compute the jacobian of that new function.
Verify that the rank of this jacobian equals the number of variables to be determined. If the number of equations is greater than the number of variables to be determined, can you identify one equation that can be safely removed?__.

In [ ]:
# Your code

@. __Write a method to solve the system in all the endogenous variables when the initial guess is close to the initial equilibrium. You can use your method of choice (using your own code, using nlsolve, ...). To ease convergence, you might want to take into account information about variable boundaries (all variables are positive).__

In [ ]:
# Your code

### Applications

@. __A massive migration doubles the endowment of labour, which is now equal to 50. Assuming structural parameters are unchanged, Compute the new equilibrium using the same code as before. Comment on the changes in quantities and prices. Could you have predicted them?__

In [2]:
# Your code

@. __What happens if the labour share in both production sectors is reduced by 1%? (i.e. $\beta_{LAB,j}$ is reduced by 1%).__

In [ ]:
# Your code

## Part II - Endogenous Exit

### Warmup: discretization of an AR1

The following code, taken from Quantecon.jl approximates an AR1 process $$y_t = \mu + \rho (y_{t-1}-\mu) + \epsilon_t$$ (where $\nu$ is the standard deviation of normal process $\epsilon$), using a finite markov chain with $N$ different values.

The output is a transition matrix $P_ij$ and a vector containing discretized values $y_1, y_2, ... y_p$

In [ ]:
# uncomment the following line if "SpecialFunctions" is not on your system
# import Pkg; Pkg.add("SpecialFunctions")

using SpecialFunctions: erfc

std_norm_cdf(x::T) where {T <: Real} = 0.5 * erfc(-x/sqrt(2))
std_norm_cdf(x::Array{T}) where {T <: Real} = 0.5 .* erfc(-x./sqrt(2))

function tauchen(N::Integer, ρ::T1, σ::T2, μ=zero(promote_type(T1, T2)), n_std::T3=3) where {T1 <: Real, T2 <: Real, T3 <: Real}
    # Get discretized space
    a_bar = n_std * sqrt(σ^2 / (1 - ρ^2))
    y = range(-a_bar, stop=a_bar, length=N)
    d = y[2] - y[1]

    # Get transition probabilities
    Π = zeros(promote_type(T1, T2), N, N)
    for row = 1:N
        # Do end points first
        Π[row, 1] = std_norm_cdf((y[1] - ρ*y[row] + d/2) / σ)
        Π[row, N] = 1 - std_norm_cdf((y[N] - ρ*y[row] - d/2) / σ)

        # fill in the middle columns
        for col = 2:N-1
            Π[row, col] = (std_norm_cdf((y[col] - ρ*y[row] + d/2) / σ) -
                           std_norm_cdf((y[col] - ρ*y[row] - d/2) / σ))
        end
    end

    yy = y .+ μ / (1 - ρ) # center process around its mean (wbar / (1 - rho)) in new variable

    (;transitions=Π, values=yy)

end

tauchen (generic function with 3 methods)

@. __Take $\rho=0.95, \mu=0.1, \nu=0.1$. Approximate the AR1 with 200 discrete states, using the tauchen function above. Check that all rows sum to 1. Compute and plot the steady-state distribution.__

In [ ]:
# you code here

@. __Simulate the evolution of the above markov chain for 10000 periods. Compute the mean and standard deviation of all the simulated values. Compare these moments with the results obtained in the preceding question.__

In [17]:
# you code here

### A firm's problem

Consider a firm whose productivity $y_t$ is exogenous and evolves according to the markov chain above.

Profits are given by $\pi(y_t) = y_t$.

At the start of each period, the firm decides whether to remain in operation and receive
current profit $\pi_t$ or to exit and receive scrap value $s>0$ for the sale of physical assets.

Time is discounted using interest rate, that is $\beta=\frac{1}{1+r} \in [0,1[$.



The following code creates a parameterization of the firm's problem:



In [ ]:
"Creates an instance of the firm exit model."
function create_exit_model(;
    n=200, # productivity grid size
    ρ=0.95, μ=0.1, ν=0.1, # persistence, mean and volatility
    β=0.98, s=100.0 # discount factor and scrap value
    )
    mc = tauchen(n, ρ, ν, μ)
    z_vals, Q = mc.state_values, mc.p
    return (; n, z_vals, Q, β, s)
end

create_exit_model

Solving the model consists in finding the value of the firm in each productivity state $y$ as well as the optimal decision (quit or continue in each of these states).

The value of the firm satisfies the Bellman equation:

$$V(y) = \max_{c(y)} \pi(y) + \beta \begin{cases} \mathbb{E}\left[ V(y') | y\right] & \text{if c(y)=true}\\ s & \text{otherwise}\end{cases}$$

Given that we have discretized the AR1 as a markov chain, one can also write it 


$$V(y_i) = \max_{c(y_i)} \pi(y_i) + \beta \begin{cases} \sum_j P_{ij} V(y_j)  & \text{if } c(y_i)=\text{true} \\ s & \text{otherwise} \end{cases}$$

For this problem it is natural to encode a guess for the value function $(V(y_i))_i$ as a `Vector{Float64}` of length $p$ and the vector of controls $(c(y_i))_i$ as a `Vector{Float64}` of length $p$.

@. __Write a function `bellman_step(model, V)` which takes in the model parameterization, as well as a guess for the next period value function ($V()$ on the right hand side of the Bellman equation) and returns a new updated value ($V()$ on the left).__

In [ ]:
# you code here

@. __Apply the `bellman_step` function until convergence, starting from any initial guess.[^vfi] Show that convergence speed is linear. Plot the resulting value function and optimal choices. Comment.__

[^vfi]: This is known as the value function iteration algorithm.

In [18]:
# you code here